In [1]:
import dspy
import pprint
from typing import List, Union

In [85]:
# Configure pprint
pp = pprint.pprint

# Set up OpenAI models
gpt4_turbo = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=300)
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300, temperature=1)

# Configure dspy settings
dspy.settings.configure(lm=gpt3_turbo, max_tokens=1024)

# Updating the core description

In [144]:
class is_collaborative(dspy.Signature):
    """Is the person trying to provide a collaborative answer to the question?"""
    question: str = dspy.InputField(desc="Question")
    answer: str = dspy.InputField(desc="The person's answer")
    assessment: bool = dspy.OutputField(desc="Is the answer being provided collaborative?")

class IsCollaborative(dspy.Module):
    """A module that decides whether the person is trying to provide a collaborative answer to the question?"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedChainOfThought(is_collaborative)

    def forward(self, question: str, answer: str) -> bool:
        return self.generate_answer(question=question, answer=answer)

Prediction(
    assessment=False
)

In [ ]:
# Buscamos False

class choose_best (dspy.Signature):
    """Choose the better description in terms of coherence, completeness of information and technicality."""
    description_a: str = dspy.InputField(desc="Description A")
    description_b: str = dspy.InputField(desc="Description B")
    assessment: bool = dspy.OutputField(desc="True if 'A' is better than 'B'")
class ChooseBest(dspy.Module):
    """A module that chooses the better description in terms of coherence, completeness of information and technicality."""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(choose_best)

    def forward(self, description_a: str, description_b: str) -> bool:
        answer = self.generate_answer(description_a=description_a, description_b=description_b)
        return answer
    
ChooseBest().forward(description_a = 'Computers are turing machines designed to follow and execute order via cycles',
                     description_b = 'Computers are turing machines designed to follow and execute order via cycles.')

In [108]:
class update_description(dspy.Signature):
    """Does minor changes to correct the description seamlessly."""
    old_description: str = dspy.InputField(desc="Original Smart Contract description")
    question: str = dspy.InputField(desc="A question to further polish the description")
    answer: str = dspy.InputField(desc="The answer provided by the client")
    new_description: List[str] = dspy.OutputField(desc="The newly writen description")

class FindIncoherences(dspy.Module):
    """A module that does minor changes to correct the description seamlessly."""
    def __init__(self, **kwargs):
        super().__init__()
        self.kwargs = kwargs
        self.generate_answer = dspy.functional.TypedChainOfThought(update_description)

    def forward(self) -> str:
        return self.generate_answer(**self.kwargs)

prompt = "We want to create a decentralized voting system where fans and nobody else can vote on various decisions related to our platform. Each fan gets one vote per decision. We want the system to be transparent, the results should be stored on the blockchain. The admin should be able to create proposals and execute them once each fan has voted two times. The system should be able to handle up to 20 votes per day. The system should be able to handle up to 40 proposals per day"
question = "Could you clarify the voting process? You mentioned each fan gets one vote per decision, but also that the admin expects each fan to vote twice. How does this work?"
answer = "You are right, I made a mistake. Fans only get one vote."

incoherences = FindIncoherences(old_description=prompt,
                                question=question,
                                answer=answer).forward()
incoherences

Prediction(
    reasoning="produce the new description. We need to correct the voting process by ensuring it's clear that each fan gets only one vote per decision, not two as previously mentioned. We also need to maintain the other requirements such as the transparency of the system, the storage of results on the blockchain, the admin's ability to create and execute proposals, and the system's capacity to handle up to 20 votes and 40 proposals per day. Finally, we must format the new description as a JSON object according to the provided schema.",
    new_description=['We aim to develop a decentralized voting system exclusively for fans to vote on various platform-related decisions. Each fan is entitled to a single vote per decision. The system will ensure transparency by recording the results on the blockchain. Administrators will have the capability to generate proposals and implement them post-voting. The system is designed to accommodate up to 20 votes and 40 proposals daily.']
)

In [93]:
# class summarize(dspy.Signature):
#     """Express the text in a conciser way, without overexplaining."""
#     text: str = dspy.InputField(desc="Some text to summarize")
#     formated_text: str = dspy.OutputField(desc="More concise text")

# class Summarize(dspy.Module):
#     """Express the text in a conciser way. Do not overexplain."""
#     def __init__(self):
#         super().__init__()
#         self.generate_answer = dspy.functional.TypedPredictor(summarize)
#     def forward(self, text: str) -> str:
#         return self.generate_answer(text=text)

In [117]:
# GPT4 recommended

prompt = "We want to create a decentralized voting system where fans and nobody else can vote on various decisions related to our platform. Each fan gets one vote per decision. We want the system to be transparent, the results should be stored on the blockchain. The admin should be able to create proposals and execute them once each fan has voted two times. The system should be able to handle up to 20 votes per day. The system should be able to handle up to 40 proposals per day"

class find_incoherences(dspy.Signature):
    """List incoherences, oddities or discrepancies in a Smart Contract Description"""
    smart_contract_description: str = dspy.InputField(desc="A description of a Smart Contract")
    incoherences: List[str] = dspy.OutputField(desc="List of incoherences (one per slot, empty if none)")

class FindIncoherences(dspy.Module):
    """A module to list incoherences, oddities or discrepancies in a Smart Contract Description"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(find_incoherences)

    def forward(self, smart_contract_description: str) -> List[str]:
        return self.generate_answer(smart_contract_description=smart_contract_description)

incoherences = FindIncoherences().forward(smart_contract_description=prompt).incoherences
incoherences

['Each fan gets one vote per decision, but the admin expects each fan to vote two times.',
 "The system's capacity is mismatched: it can handle 20 votes per day but needs to support up to 40 proposals per day."]

In [120]:
class inquire_about_incoherence(dspy.Signature):
    """
    Spots an incoherence and asks for clarification.
    Keep it as short as possible.
    """
    smart_contract_description: str = dspy.InputField(desc="A description of a Smmart Contract")
    specific_incoherence: str = dspy.InputField(desc="The incohrence to be clarified")
    conversational_question: str = dspy.OutputField(desc="A question formulated targeting the incoherence")

class InquireAboutIncoherence(dspy.Module):
    """
    Spots an incoherence and asks for clarification.
    Keep it as short as possible.
    """
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(inquire_about_incoherence)
        
    def forward(self, smart_contract_description: str, specific_incoherence: str) -> str:
        return self.generate_answer(smart_contract_description=smart_contract_description, specific_incoherence=specific_incoherence)

qs = []
for incoherence in incoherences:
    print(f"Incoherence: {incoherence}")
    q = InquireAboutIncoherence().forward(smart_contract_description=prompt, specific_incoherence=incoherence)
    print(f"   Question: {question}")
    qs.append(q)


Incoherence: Each fan gets one vote per decision, but the admin expects each fan to vote two times.
   Question: Could you clarify the voting process? You mentioned each fan gets one vote per decision, but also that the admin expects each fan to vote twice. How does this work?
Incoherence: The system's capacity is mismatched: it can handle 20 votes per day but needs to support up to 40 proposals per day.
   Question: Could you clarify the voting process? You mentioned each fan gets one vote per decision, but also that the admin expects each fan to vote twice. How does this work?


In [127]:
class inquire_insights(dspy.Signature):
    """
    Make a single specific question to further dig into the technical aspects of the description.
    You can ask for clarifications, details, measures and/or concrete digits.
    Provide possible solutions whenever possible.
    """
    smart_contract_description: str = dspy.InputField(desc="A description of a Smmart Contract")
    conversational_question: str = dspy.OutputField(desc="A single question targeting technical aspects")

class InquireInsights(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(inquire_insights)
        
    def forward(self, smart_contract_description: str) -> str:
        return self.generate_answer(smart_contract_description=smart_contract_description)
    
InquireInsights().forward(smart_contract_description=prompt)

Prediction(
    conversational_question='Conversational Question: Could you clarify the requirement for each fan to vote "two times" on a proposal? Does this mean they are allowed to change their vote once, or are there two rounds of voting for each proposal, and what is the intended mechanism to ensure a fan votes exactly twice before the admin can execute the proposal?'
)